In [ ]:
import torch
import numpy as np
import math
from functools import partial
import torch.nn as nn
import torch.nn.functional as F

import tensorflow as tf

In [ ]:
print(torch.__version__)

1.10.0+cu111


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math
from functools import partial

__all__ = [
    'ResNet', 'resnet10', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
    'resnet152', 'resnet200'
]


def conv3x3x3(in_planes, out_planes, stride=1):
    # 3x3x3 convolution with padding
    return nn.Conv3d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=1,
        bias=False)


def downsample_basic_block(x, planes, stride):
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    zero_pads = torch.Tensor(
        out.size(0), planes - out.size(1), out.size(2), out.size(3),
        out.size(4)).zero_()
    if isinstance(out.data, torch.cuda.FloatTensor):
        zero_pads = zero_pads.cuda()

    out = Variable(torch.cat([out.data, zero_pads], dim=1))

    return out


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = nn.Conv3d(
            planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = nn.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self,
                 block,
                 layers,
                 sample_size,
                 sample_duration,
                 shortcut_type='B',
                 num_classes=400):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv3d(
            3,
            64,
            kernel_size=7,
            stride=(1, 2, 2),
            padding=(3, 3, 3),
            bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], shortcut_type)
        self.layer2 = self._make_layer(
            block, 128, layers[1], shortcut_type, stride=2)
        self.layer3 = self._make_layer(
            block, 256, layers[2], shortcut_type, stride=2)
        self.layer4 = self._make_layer(
            block, 512, layers[3], shortcut_type, stride=2)
        last_duration = int(math.ceil(sample_duration / 16))
        last_size = int(math.ceil(sample_size / 32))
        self.avgpool = nn.AvgPool3d(
            (last_duration, last_size, last_size), stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(
                    downsample_basic_block,
                    planes=planes * block.expansion,
                    stride=stride)
            else:
                downsample = nn.Sequential(
                    nn.Conv3d(
                        self.inplanes,
                        planes * block.expansion,
                        kernel_size=1,
                        stride=stride,
                        bias=False), nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        #x = self.fc(x) -> we keep only the features extractor without the classification layer

        return x


def get_fine_tuning_parameters(model, ft_begin_index):
    if ft_begin_index == 0:
        return model.parameters()

    ft_module_names = []
    for i in range(ft_begin_index, 5):
        ft_module_names.append('layer{}'.format(i))
    ft_module_names.append('fc')

    parameters = []
    for k, v in model.named_parameters():
        for ft_module in ft_module_names:
            if ft_module in k:
                parameters.append({'params': v})
                break
        else:
            parameters.append({'params': v, 'lr': 0.0})

    return parameters


def resnet10(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [1, 1, 1, 1], **kwargs)
    return model


def resnet18(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model


def resnet34(**kwargs):
    """Constructs a ResNet-34 model.
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    return model


def resnet50(**kwargs):
    """Constructs a ResNet-50 model.
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model


def resnet101(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    return model


def resnet152(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    return model


def resnet200(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 24, 36, 3], **kwargs)
    return model


In [ ]:
#instance du modèle ResNet34-3D
resnet34_model = resnet34(sample_size = 112 , sample_duration = 16)

#pre-trained model
PATH = 'resnet-34-kinetics.pth'
pretrain = torch.load(PATH , map_location = 'cpu')

#charge dans notre instance Resnet34 les poids pre-trained
resnet34_model.load_state_dict(pretrain['state_dict'], strict=False)
resnet34_model.eval()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:145: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


ResNet(
  (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [ ]:
print("Model's state_dict:")
for param_tensor in resnet34_model.state_dict():
    print(param_tensor, "\t", resnet34_model.state_dict()[param_tensor].size())

Model's state_dict:
conv1.weight 	 torch.Size([64, 3, 7, 7, 7])
bn1.weight 	 torch.Size([64])
bn1.bias 	 torch.Size([64])
bn1.running_mean 	 torch.Size([64])
bn1.running_var 	 torch.Size([64])
bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv1.weight 	 torch.Size([64, 64, 3, 3, 3])
layer1.0.bn1.weight 	 torch.Size([64])
layer1.0.bn1.bias 	 torch.Size([64])
layer1.0.bn1.running_mean 	 torch.Size([64])
layer1.0.bn1.running_var 	 torch.Size([64])
layer1.0.bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv2.weight 	 torch.Size([64, 64, 3, 3, 3])
layer1.0.bn2.weight 	 torch.Size([64])
layer1.0.bn2.bias 	 torch.Size([64])
layer1.0.bn2.running_mean 	 torch.Size([64])
layer1.0.bn2.running_var 	 torch.Size([64])
layer1.0.bn2.num_batches_tracked 	 torch.Size([])
layer1.1.conv1.weight 	 torch.Size([64, 64, 3, 3, 3])
layer1.1.bn1.weight 	 torch.Size([64])
layer1.1.bn1.bias 	 torch.Size([64])
layer1.1.bn1.running_mean 	 torch.Size([64])
layer1.1.bn1.running_var 	 torch.Size([64])
layer1.1.

In [ ]:
#inputs formats: (batch_size, num_channels, sample_duration, size_frame)

#test rapide du resnet34
inputs = np.ones((1,3,16,112,112), dtype=np.float32) #must specified float32

inputs = torch.from_numpy(inputs)
outputs = resnet34_model(inputs)

outputs.shape

torch.Size([1, 512])

In [ ]:
#test utilisation du model resnet34 pytorch avec un reseau tensorflow

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(10 ,input_shape=(512,),  activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                5130      
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,141
Trainable params: 5,141
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#base de données simulées avec 3 samples

train_data = np.ones((3,3,16,112,112), dtype=np.float32) #must specified float32
train_labels = np.array([[11.],[112.],[0.3]], dtype=np.float32)



In [ ]:
loss_object = tf.keras.losses.mse #loss mse juste pour tester la backpropa
sgd = tf.keras.optimizers.Adam()

def train_step(data, labels, model, resnet34):
    
    #essayer le Resnet ici avant le GradientTape

    #convert data to torch tensor
    inputs = torch.from_numpy(data)
    print(inputs.shape)
    features512D = resnet34(inputs)
    features512D = features512D.cpu().detach().numpy() #-> numpy only supported by cpu

    #tf.GradientTape -> enregistrer les opérations de propagation pour les réutiliser
    #lors de la rétropropagation
    with tf.GradientTape() as tape:
        
        #convert data to torch tensor
        #inputs = torch.from_numpy(data)
        #print(inputs.shape)
        #features512D = resnet34(inputs)#, trainable=False)
        #features512D = features512D.cpu().detach().numpy() #-> numpy only supported by cpu
        
        # forward pass: input as numpy -> by default TF accept it
        predictions = model(features512D)

        #calcul de la loss
        loss = loss_object(labels, predictions)

    # calcul des gradients par rapport aux variables/parametres/poids du modèle
    gradient = tape.gradient(loss, model.trainable_variables)

    # retropropagation
    sgd.apply_gradients(zip(gradient, model.trainable_variables))

    #loss pour un batch, one step
    return loss

train_step(train_data, train_labels, model, resnet34_model)

torch.Size([3, 3, 16, 112, 112])


<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1.0000000e+02, 1.2321000e+04, 4.8999998e-01], dtype=float32)>

In [ ]:
#boucle d'apprentissage

def train(epochs, batch_size, model, resnet34):

  for epoch in range(epochs):

    print('epoch : ', epoch+1,'/',epochs)

    for i in range (0, len(train_data), batch_size):

      #batch
      batch_data = train_data[i:i+batch_size]
      batch_labels = train_labels[i:i+batch_size]

      #forward-backpropa
      loss = train_step(batch_data, batch_labels, model, resnet34_model)
    
    print('Loss : ', np.sum(loss.numpy()))

train(10,3,model,resnet34_model)

epoch :  1 / 10
torch.Size([3, 3, 16, 112, 112])
Loss :  12421.49
epoch :  2 / 10
torch.Size([3, 3, 16, 112, 112])
Loss :  12421.49
epoch :  3 / 10
torch.Size([3, 3, 16, 112, 112])
Loss :  12421.49
epoch :  4 / 10
torch.Size([3, 3, 16, 112, 112])
Loss :  12421.49
epoch :  5 / 10
torch.Size([3, 3, 16, 112, 112])
Loss :  12421.49
epoch :  6 / 10
torch.Size([3, 3, 16, 112, 112])
Loss :  12421.49
epoch :  7 / 10
torch.Size([3, 3, 16, 112, 112])
Loss :  12421.49
epoch :  8 / 10
torch.Size([3, 3, 16, 112, 112])
Loss :  12421.49
epoch :  9 / 10
torch.Size([3, 3, 16, 112, 112])
Loss :  12421.49
epoch :  10 / 10
torch.Size([3, 3, 16, 112, 112])
Loss :  12421.49
